<a href="https://colab.research.google.com/github/samueleallen/Scraping-Val-Data/blob/main/scraping_edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [310]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
from google.colab import files

In [364]:
# scrape team urls from standings page
standings_url = 'https://www.vlr.gg/vct-2024/standings'
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)

In [366]:
# Locate team urls
standings_table = soup.select('div.eg-standing-container')[0]
links = standings_table.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if '/team/' in l]
# Format link
team_urls = [f'https://www.vlr.gg{l}' for l in links]

['https://www.vlr.gg/team/2359/leviat-n', 'https://www.vlr.gg/team/11058/g2-esports', 'https://www.vlr.gg/team/2/sentinels', 'https://www.vlr.gg/team/120/100-thieves', 'https://www.vlr.gg/team/2355/kr-esports', 'https://www.vlr.gg/team/188/cloud9', 'https://www.vlr.gg/team/5248/evil-geniuses', 'https://www.vlr.gg/team/1034/nrg-esports', 'https://www.vlr.gg/team/6961/loud', 'https://www.vlr.gg/team/2406/furia', 'https://www.vlr.gg/team/7386/mibr', 'https://www.vlr.gg/team/17/gen-g', 'https://www.vlr.gg/team/624/paper-rex', 'https://www.vlr.gg/team/8185/drx', 'https://www.vlr.gg/team/6199/team-secret', 'https://www.vlr.gg/team/8304/talon', 'https://www.vlr.gg/team/878/rex-regum-qeon', 'https://www.vlr.gg/team/14/t1', 'https://www.vlr.gg/team/918/global-esports', 'https://www.vlr.gg/team/5448/zeta-division', 'https://www.vlr.gg/team/278/detonation-focusme', 'https://www.vlr.gg/team/6387/bleed', 'https://www.vlr.gg/team/1184/fut-esports', 'https://www.vlr.gg/team/8877/karmine-corp', 'https

In [373]:
# CSV file path for team map stats
csv_file_path = '/content/team_map_stats.csv'

# List to hold all team data
all_team_data = []

for team_url in team_urls: # just get rid of [:1] to fix loop later, did this so runtime is not too long
  data = requests.get(team_url)

  # Scrape stats section data from team page
  soup = BeautifulSoup(data.text)
  links = soup.find_all('a')
  links = [l.get("href") for l in links]
  links = [l for l in links if l and 'team/stats' in l]

  for link in links:
    data = requests.get(f"https://www.vlr.gg{link}")
    soup = BeautifulSoup(data.text)  # Create BeautifulSoup object for stats page
    table = soup.find('table', class_='wf-table mod-team-maps') # find table on webpage
    # Find the h1 element with class 'wf-title' and get the text
    team_name = soup.find('h1', class_='wf-title').get_text(strip=True)
    # Use pandas to read the table
    stats = pd.read_html(str(table))[0]
    # Filter rows to only be rows that start with specific map names
    map_names = ["Sunset", "Bind", "Haven", "Split", "Ascent", "Icebox", "Breeze", "Fracture", "Pearl", "Lotus", "Abyss"]
    filtered_stats = stats[stats.iloc[:, 0].str.startswith(tuple(map_names), na=False)]
    filtered_stats = filtered_stats.drop(columns=["Agent Compositions", "Expand"])
    filtered_stats['Team'] = team_name
    all_team_data.append(filtered_stats)
  # After the loop finishes, concatenate all the team data into a single DataFrame
if all_team_data:
    full_team_stats = pd.concat(all_team_data, ignore_index=True)

    # Write the combined team stats to the CSV file (overwrite the file)
    full_team_stats.to_csv(csv_file_path, mode='w', header=True, index=False)

    # After the loop finishes, download the file
    # files.download(csv_file_path)

<ipython-input-373-93d9d889d6fe>:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]


          Map (#) WIN%   W   L ATK 1st DEF 1st ATK RWin%   RW   RL DEF RWin%  \
0       Bind (33)  42%  14  19      12      21       46%  154  184       53%   
34     Haven (42)  60%  25  17      13      29       58%  253  185       45%   
77     Split (27)  41%  11  16      10      17       49%  133  140       49%   
105   Ascent (47)  53%  25  22      23      24       47%  217  248       58%   
153   Icebox (35)  74%  26   9      11      24       56%  191  149       55%   
189   Breeze (21)  57%  12   9       8      13       54%  114   97       56%   
211  Fracture (8)  50%   4   4       1       7       54%   35   30       47%   
220    Pearl (12)  75%   9   3       2      10       56%   67   53       55%   
233    Lotus (22)  55%  12  10      10      12       58%  132   96       40%   
256   Sunset (13)  69%   9   4       3      10       61%   69   45       51%   
270     Abyss (5)  80%   4   1       2       3       46%   25   29       66%   

    RW.1 RL.1      Team  
0    194  173

<ipython-input-373-93d9d889d6fe>:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]


          Map (#) WIN%   W   L ATK 1st DEF 1st ATK RWin%   RW   RL DEF RWin%  \
0       Bind (23)  48%  11  12      10      13       57%  135  102       47%   
24     Haven (21)  43%   9  12       8      13       46%  104  122       49%   
46     Split (20)  65%  13   7      13       7       56%  121   97       52%   
67    Ascent (23)  61%  14   9      14       9       48%  131  141       51%   
91    Icebox (21)  48%  10  11       9      12       54%  104   89       48%   
113    Breeze (8)  50%   4   4       5       3       48%   44   47       51%   
122  Fracture (5)  40%   2   3       2       3       56%   24   19       35%   
128     Pearl (7)  71%   5   2       1       6       66%   40   21       43%   
136    Lotus (24)  63%  15   9      11      13       51%  125  119       55%   
161   Sunset (11)  36%   4   7       4       7       52%   59   54       48%   
173     Abyss (5)  60%   3   2       2       3       52%   28   26       50%   

    RW.1 RL.1        Team  
0    113  1

<ipython-input-373-93d9d889d6fe>:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]


...

<ipython-input-373-93d9d889d6fe>:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]


         Map (#)  WIN%  W  L ATK 1st DEF 1st ATK RWin%  RW  RL DEF RWin% RW.1  \
0      Bind (11)   36%  4  7       6       5       46%  58  67       46%   55   
12     Haven (8)  100%  8  0       2       6       66%  50  26       62%   54   
21    Split (10)   40%  4  6       5       5       44%  41  52       55%   51   
32   Ascent (14)   64%  9  5       8       6       43%  68  91       60%   91   
47    Icebox (6)   67%  4  2       4       2       61%  43  27       49%   32   
54    Breeze (2)   50%  1  1       0       2       45%  10  12       58%   14   
57  Fracture (2)   50%  1  1       1       1       48%  11  12       54%   13   
60     Pearl (3)  100%  3  0       0       3       48%  14  15       69%   25   
64     Lotus (7)   29%  2  5       5       2       45%  34  42       51%   37   
72   Sunset (11)   45%  5  6       3       8       45%  48  59       52%   67   
84     Abyss (9)   56%  5  4       6       3       53%  52  47       54%   44   

   RL.1            Team  
0

<ipython-input-373-93d9d889d6fe>:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]


          Map (#) WIN%   W   L ATK 1st DEF 1st ATK RWin%   RW   RL DEF RWin%  \
0       Bind (20)  45%   9  11       8       9       46%   79   94       48%   
21      Haven (9)  22%   2   7       2       7       48%   40   44       45%   
31     Split (29)  55%  16  13      21       5       52%  154  144       49%   
61    Ascent (24)  50%  12  12      13      11       47%  123  137       54%   
86    Icebox (10)  60%   6   4       4       6       60%   59   40       42%   
97     Breeze (0)    -   0   0       0       0         -    0    0         -   
98   Fracture (4)  50%   2   2       2       2       62%   29   18       44%   
103    Pearl (10)  20%   2   8       2       8       46%   41   49       43%   
114    Lotus (17)  29%   5  12       6      10       47%   76   85       42%   
132   Sunset (17)  59%  10   7       8       8       56%   84   66       59%   
150    Abyss (10)  80%   8   2       4       6       53%   52   46       64%   

    RW.1 RL.1          Team  
0     82 

<ipython-input-373-93d9d889d6fe>:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]


In [370]:
# CSV file path for player stats
csv_file_path = '/content/player_stats.csv'

# List to hold all player data
all_player_data = []

# Fetches individual player data from their entire career
for team_url in team_urls:
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text)
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'player' in l]

    # Convert to full player URLs and filter to only players, not coaching & staff
    player_urls = [f"https://www.vlr.gg{l}" for l in links]
    player_stats_urls = [f"{l}/?timespan=all" for l in player_urls[:5]]

    # Loop through each player and collect their data
    for player_url in player_stats_urls:
        try:
            data = requests.get(player_url)
            soup = BeautifulSoup(data.text)
            table = soup.find('table', class_='wf-table')

            if table is None:
                print(f"No table found for {player_url}")
                continue  # Skip this player and move to the next

            # Read table with pandas
            player_stats = pd.read_html(str(table))[0]

            # Extract player name from URL
            player_name = player_url.split("/")[-2]  # Gets the player name from the URL

            # Extract agent names from images
            agent_names = []
            for img_tag in table.find_all('img'):
                img_src = img_tag.get('src')
                agent_name = os.path.splitext(os.path.basename(img_src))[0]
                agent_names.append(agent_name)

            # Add extracted agent names and player name column
            player_stats['Agent'] = agent_names
            player_stats['Player'] = player_name  # Append player name column
            player_stats = player_stats.drop(columns=["Unnamed: 0"])

            # Append player stats to the all_player_data list
            all_player_data.append(player_stats)

        except Exception as e:
            print(f"Error processing {player_url}: {e}")

# After the loop finishes, concatenate all the player data into a single DataFrame
if all_player_data:
    full_stats = pd.concat(all_player_data, ignore_index=True)

    # Write the combined player stats to the CSV file (overwrite the file)
    full_stats.to_csv(csv_file_path, mode='w', header=True, index=False)


<ipython-input-370-9b8e19463320>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-370-9b8e19463320>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-370-9b8e19463320>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-370-9b8e19463320>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))

No table found for https://www.vlr.gg/player/40862/benedict/?timespan=all
No table found for https://www.vlr.gg/player/31688/impression/?timespan=all
No table found for https://www.vlr.gg/390246/bleed-announce-roster-overhaul-bench-four-players/?timespan=all


<ipython-input-370-9b8e19463320>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-370-9b8e19463320>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-370-9b8e19463320>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-370-9b8e19463320>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))

In [360]:
# CSV file path for player stats
csv_file_path = '/content/player_stats_90days.csv'

# List to hold all player data
all_player_data = []

# Fetches individual player data from their entire career
for team_url in team_urls:
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text)
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'player' in l]

    # Convert to full player URLs and filter to only players, not coaching & staff
    player_urls = [f"https://www.vlr.gg{l}" for l in links]
    player_stats_urls = [f"{l}/?timespan=90d" for l in player_urls[:5]]

    # Loop through each player and collect their data
    for player_url in player_stats_urls:
        try:
            data = requests.get(player_url)
            soup = BeautifulSoup(data.text)
            table = soup.find('table', class_='wf-table')

            if table is None:
                print(f"No table found for {player_url}")
                continue  # Skip this player and move to the next

            # Read table with pandas
            player_stats = pd.read_html(str(table))[0]

            # Extract player name from URL
            player_name = player_url.split("/")[-2]  # Gets the player name from the URL

            # Extract agent names from images
            agent_names = []
            for img_tag in table.find_all('img'):
                img_src = img_tag.get('src')
                agent_name = os.path.splitext(os.path.basename(img_src))[0]
                agent_names.append(agent_name)

            # Add extracted agent names and player name column
            player_stats['Agent'] = agent_names
            player_stats['Player'] = player_name  # Append player name column
            player_stats = player_stats.drop(columns=["Unnamed: 0"])

            # Append player stats to the all_player_data list
            all_player_data.append(player_stats)

        except Exception as e:
            print(f"Error processing {player_url}: {e}")

# After the loop finishes, concatenate all the player data into a single DataFrame
if all_player_data:
    full_stats = pd.concat(all_player_data, ignore_index=True)

    # Write the combined player stats to the CSV file (overwrite the file)
    full_stats.to_csv(csv_file_path, mode='w', header=True, index=False)

    # After the loop finishes, download the file
    # files.download(csv_file_path)


<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))

No table found for https://www.vlr.gg/player/40862/benedict/?timespan=90d
No table found for https://www.vlr.gg/player/31688/impression/?timespan=90d
No table found for https://www.vlr.gg/390246/bleed-announce-roster-overhaul-bench-four-players/?timespan=90d


<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))

No table found for https://www.vlr.gg/player/37020/xiyiji/?timespan=90d


<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-360-e2f5968ef606>:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))

In [316]:
# I have all-time player stats, past 90 days player stats, team map stats, now we need match up stats for teams 😸
# Ex: fnatic vs sentinels, sentinels has won x out of y matchups
matches_url = 'https://www.vlr.gg/matches'
data = requests.get(matches_url)
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'page' in l]
url = "https://www.vlr.gg/matches/results"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [317]:
# Find all page number links
page_links = soup.find_all("a", class_="btn mod-page")
# Page_links holds a list of HTML <a> elements beautiful soup found
# Now extract the page numbers
page_numbers = [int(link.text) for link in page_links if link.text.isdigit()]
outer_list = []

# Get the max page number
max_page = max(page_numbers)

In [406]:
for page in range(1, max_page + 1): # For loop starts from page 1 ends at max page
# FIX LOOP BOUNDS LATER 🐱‍🐉🐱‍🐉🐱‍🐉🐱‍🐉🐱‍🐉
# also implement max wait time with loops to help with run time
    url = f"https://www.vlr.gg/matches/results/?page={page}"
    data = requests.get(url)
    soup = BeautifulSoup(data.text, "html.parser")
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    # Filter links to only include recent years, champions tour matches, and not acension champions tour matches
    links = [l for l in links if l and 'champions-tour' in l and ('2023' in l or '2024' in l or '2025' in l) and 'ascension' not in l and 'challengers' not in l]
    matches = [f"https://www.vlr.gg{l}" for l in links]
    # append each link to a different list so all elements stay in one list throughout the 550+ loops
    outer_list.extend(matches)

In [425]:
import re

# CSV file path for overall game stats
csv_file_path = '/content/overall_game_stats.csv'

# List to hold all player data
all_game_stats = []

# Outer list holds every vct match from 2023-current time
for match in outer_list:
    try:
        data = requests.get(match)
        soup = BeautifulSoup(data.text)  # Create BeautifulSoup object for stats page
        table = soup.find_all('table', class_='wf-table-inset mod-overview')
        team_names = [team.get_text(strip=True) for team in soup.find_all('div', class_='wf-title-med')]

        # Extract the year from the event name
        event_string = soup.find('div', style='font-weight: 700;').get_text(strip=True)  # Find the event string
        match_year = re.search(r'\d{4}', event_string)  # Find the year in the event string
        if match_year:
            year = match_year.group(0)
        else:
            year = None  # If no year is found, set it as None

        # Extract the winner and loser scores
        winner_score = soup.find('span', class_='match-header-vs-score-winner').get_text(strip=True)
        loser_score = soup.find('span', class_='match-header-vs-score-loser').get_text(strip=True)
        print("Winner score:", winner_score)
        print("Loser score:", loser_score)

        # Extract the match date from the div with class 'moment-tz-convert'
        date_string = soup.find('div', class_='moment-tz-convert')['data-utc-ts']  # Extract the full timestamp
        date = pd.to_datetime(date_string).strftime('%Y-%m-%d')  # Convert to desired date format (YYYY-MM-DD)

        # j = 0 will always be the winning team due to the way the website works
        result = 'Win'  # Team 0 is always the winner
        final_score = f"{winner_score}-{loser_score}"

        # Use pandas to read the table
        for j in range(0, 2):
            try:
                all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
                all_map_stats = all_map_stats.drop(columns=["Unnamed: 1"])

                # Clean the data by keeping only the first number in each cell
                all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)

                # Add the year and date to the DataFrame
                all_map_stats['date'] = date
                all_map_stats['Team'] = team_names[j]
                all_map_stats['vs Team'] = team_names[1-j]

                # Set the result column based on the winning team (team 0 is the winner)
                if j == 0:
                    all_map_stats['result'] = 'Win'
                    all_map_stats['final_score'] = f"'{winner_score} : {loser_score}'"  # Winner score comes first
                else:
                    all_map_stats['result'] = 'Loss'
                    all_map_stats['final_score'] = f"'{loser_score} : {winner_score}'"  # Loser score comes first

                # Rename columns as needed
                all_map_stats = all_map_stats.rename(columns={'Unnamed: 0': 'Player', '+/–': '+/- K/D', '+/–.1': '+/- FK/FD'})

                all_game_stats.append(all_map_stats)

            except Exception as e:
                print(f"Error processing match data for {match}, table {j}: {e}")
                continue  # Skip this iteration and continue with the next table

    except Exception as e:
        print(f"Error processing match {match}: {e}")
        continue  # Skip this iteration and continue with the next match

# After the loop finishes, concatenate all the player data into a single DataFrame
if all_game_stats:
    full_stats = pd.concat(all_game_stats, ignore_index=True)

    # Write the combined game stats to the CSV file (overwrite the file)
    full_stats.to_csv(csv_file_path, mode='w', header=True, index=False)

    # Optionally, after the loop finishes, download the file
    # files.download(csv_file_path)


Winner score: 3
Loser score: 2


<ipython-input-425-d44711bc5f24>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-425-d44711bc5f24>:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)
<ipython-input-425-d44711bc5f24>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-425-d44711bc5f24>:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stat

Winner score: 3
Loser score: 2


<ipython-input-425-d44711bc5f24>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-425-d44711bc5f24>:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)
<ipython-input-425-d44711bc5f24>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables


Winner score: 3
Loser score: 2


...

<ipython-input-425-d44711bc5f24>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-425-d44711bc5f24>:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)
<ipython-input-425-d44711bc5f24>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-425-d44711bc5f24>:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stat

Winner score: 2
Loser score: 0


<ipython-input-425-d44711bc5f24>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-425-d44711bc5f24>:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)
<ipython-input-425-d44711bc5f24>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-425-d44711bc5f24>:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stat

Winner score: 2
Loser score: 0


<ipython-input-425-d44711bc5f24>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-425-d44711bc5f24>:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stats = all_map_stats.applymap(lambda x: str(x).lstrip('/ ').split()[0] if isinstance(x, str) and len(str(x).split()) > 0 else x)
<ipython-input-425-d44711bc5f24>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_map_stats = pd.read_html(str(table))[j]  # max index is 1, min index is 0, need both tables
<ipython-input-425-d44711bc5f24>:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_map_stat

In [320]:
# minor error, table gets confusing. Elements of three which go from these stats categories "all -> attack -> defend"
# Need to get data href for each map played
# And need to get each data-game id
# The next bit of code is to try to get each map's stats for each map played in a game. Previously,
# we got the stats for all the maps combined but we want to see how they performed on each map.
data = requests.get(outer_list[0]) # will need a for loop later to go through every element of outer list 🐈

In [321]:
# Locate team urls
# maps = soup.select('vm-stats-gamesnav-item js-map-switch')[0]
soup = BeautifulSoup(data.text, 'html.parser')
# need amount of maps played for max index range
divs = soup.find_all('div', class_='vm-stats-gamesnav-item js-map-switch')
all_match_tabs = []

In [322]:
# Find all div elements with the given class
for i in range(0, len(divs)):
  divs = soup.find_all('div', class_='vm-stats-gamesnav-item js-map-switch')[i]
  # Extract and print the 'data-href' attributes
  data_href = divs.get('data-href')
  game_id = divs.get("data-game-id")
  if data_href and game_id:
    final_url = f"https://www.vlr.gg{data_href}&game={game_id}&tab=overview"
    all_match_tabs.append(final_url)
    print(all_match_tabs)

['https://www.vlr.gg/429390/team-vitality-vs-team-liquid-champions-tour-2025-emea-kickoff-gf/?map=1&game=196100&tab=overview']
['https://www.vlr.gg/429390/team-vitality-vs-team-liquid-champions-tour-2025-emea-kickoff-gf/?map=1&game=196100&tab=overview', 'https://www.vlr.gg/429390/team-vitality-vs-team-liquid-champions-tour-2025-emea-kickoff-gf/?map=2&game=196101&tab=overview']
['https://www.vlr.gg/429390/team-vitality-vs-team-liquid-champions-tour-2025-emea-kickoff-gf/?map=1&game=196100&tab=overview', 'https://www.vlr.gg/429390/team-vitality-vs-team-liquid-champions-tour-2025-emea-kickoff-gf/?map=2&game=196101&tab=overview', 'https://www.vlr.gg/429390/team-vitality-vs-team-liquid-champions-tour-2025-emea-kickoff-gf/?map=3&game=196102&tab=overview']
['https://www.vlr.gg/429390/team-vitality-vs-team-liquid-champions-tour-2025-emea-kickoff-gf/?map=1&game=196100&tab=overview', 'https://www.vlr.gg/429390/team-vitality-vs-team-liquid-champions-tour-2025-emea-kickoff-gf/?map=2&game=196101&tab

In [352]:
for i in range(0, (len(all_match_tabs))): # outer range should be len(all_match_tabs) when done which is 5 maps current, usually 3
  data = requests.get(all_match_tabs[i])
  soup = BeautifulSoup(data.text)  # Create BeautifulSoup object for stats page
  table = soup.find_all('table', class_='wf-table-inset mod-overview')
  individual_map_stats_team = pd.read_html(str(table))[i]
  individual_map_stats_team = individual_map_stats_team.drop(columns=["Unnamed: 1"])
  print(individual_map_stats_team.head())


<ipython-input-352-17e2ae8b064b>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  individual_map_stats_team = pd.read_html(str(table))[0]


  Unnamed: 0            R2.0          ACS        K            D      A  \
0  Derke VIT  1.23 0.38 2.02  287 159 405  20 5 15  / 19 10 9 /  4 2 2   
1   Less VIT  1.14 2.15 0.22   227 387 82  20 17 3  / 17 6 11 /  5 3 2   
2  trexx VIT  0.63 0.68 0.59  161 180 145   12 7 5  / 19 9 10 /  4 2 2   
3  Kicks VIT  0.62 0.96 0.30   103 154 58    9 6 3  / 16 6 10 /  7 5 2   
4   Sayf VIT  0.37 0.32 0.42   112 97 126    9 4 5  / 20 9 11 /  4 2 2   

         +/–         KAST          ADR          HS%     FK     FD     +/–.1  
0   +1 -5 +6  78% 73% 83%  179 118 235  35% 28% 38%  8 2 6  4 3 1  +4 -1 +5  
1  +3 +11 -8  78% 91% 67%   140 235 53  32% 32% 33%  0 0 0  1 0 1   -1 0 -1  
2   -7 -2 -5  70% 91% 50%  120 139 103  17% 23% 13%  1 1 0  4 3 1  -3 -2 -1  
3    -7 0 -7  65% 82% 50%    66 105 29  25% 19% 67%  0 0 0  2 1 1  -2 -1 -1  
4  -11 -5 -6  52% 64% 42%     70 56 83  29% 44% 23%  2 1 1  1 0 1   +1 +1 0  


<ipython-input-352-17e2ae8b064b>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  individual_map_stats_team = pd.read_html(str(table))[0]


  Unnamed: 0            R2.0          ACS        K            D      A  \
0  Derke VIT  1.23 0.38 2.02  287 159 405  20 5 15  / 19 10 9 /  4 2 2   
1   Less VIT  1.14 2.15 0.22   227 387 82  20 17 3  / 17 6 11 /  5 3 2   
2  trexx VIT  0.63 0.68 0.59  161 180 145   12 7 5  / 19 9 10 /  4 2 2   
3  Kicks VIT  0.62 0.96 0.30   103 154 58    9 6 3  / 16 6 10 /  7 5 2   
4   Sayf VIT  0.37 0.32 0.42   112 97 126    9 4 5  / 20 9 11 /  4 2 2   

         +/–         KAST          ADR          HS%     FK     FD     +/–.1  
0   +1 -5 +6  78% 73% 83%  179 118 235  35% 28% 38%  8 2 6  4 3 1  +4 -1 +5  
1  +3 +11 -8  78% 91% 67%   140 235 53  32% 32% 33%  0 0 0  1 0 1   -1 0 -1  
2   -7 -2 -5  70% 91% 50%  120 139 103  17% 23% 13%  1 1 0  4 3 1  -3 -2 -1  
3    -7 0 -7  65% 82% 50%    66 105 29  25% 19% 67%  0 0 0  2 1 1  -2 -1 -1  
4  -11 -5 -6  52% 64% 42%     70 56 83  29% 44% 23%  2 1 1  1 0 1   +1 +1 0  


<ipython-input-352-17e2ae8b064b>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  individual_map_stats_team = pd.read_html(str(table))[0]


  Unnamed: 0            R2.0          ACS        K            D      A  \
0  Derke VIT  1.23 0.38 2.02  287 159 405  20 5 15  / 19 10 9 /  4 2 2   
1   Less VIT  1.14 2.15 0.22   227 387 82  20 17 3  / 17 6 11 /  5 3 2   
2  trexx VIT  0.63 0.68 0.59  161 180 145   12 7 5  / 19 9 10 /  4 2 2   
3  Kicks VIT  0.62 0.96 0.30   103 154 58    9 6 3  / 16 6 10 /  7 5 2   
4   Sayf VIT  0.37 0.32 0.42   112 97 126    9 4 5  / 20 9 11 /  4 2 2   

         +/–         KAST          ADR          HS%     FK     FD     +/–.1  
0   +1 -5 +6  78% 73% 83%  179 118 235  35% 28% 38%  8 2 6  4 3 1  +4 -1 +5  
1  +3 +11 -8  78% 91% 67%   140 235 53  32% 32% 33%  0 0 0  1 0 1   -1 0 -1  
2   -7 -2 -5  70% 91% 50%  120 139 103  17% 23% 13%  1 1 0  4 3 1  -3 -2 -1  
3    -7 0 -7  65% 82% 50%    66 105 29  25% 19% 67%  0 0 0  2 1 1  -2 -1 -1  
4  -11 -5 -6  52% 64% 42%     70 56 83  29% 44% 23%  2 1 1  1 0 1   +1 +1 0  


<ipython-input-352-17e2ae8b064b>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  individual_map_stats_team = pd.read_html(str(table))[0]


  Unnamed: 0            R2.0          ACS        K            D      A  \
0  Derke VIT  1.23 0.38 2.02  287 159 405  20 5 15  / 19 10 9 /  4 2 2   
1   Less VIT  1.14 2.15 0.22   227 387 82  20 17 3  / 17 6 11 /  5 3 2   
2  trexx VIT  0.63 0.68 0.59  161 180 145   12 7 5  / 19 9 10 /  4 2 2   
3  Kicks VIT  0.62 0.96 0.30   103 154 58    9 6 3  / 16 6 10 /  7 5 2   
4   Sayf VIT  0.37 0.32 0.42   112 97 126    9 4 5  / 20 9 11 /  4 2 2   

         +/–         KAST          ADR          HS%     FK     FD     +/–.1  
0   +1 -5 +6  78% 73% 83%  179 118 235  35% 28% 38%  8 2 6  4 3 1  +4 -1 +5  
1  +3 +11 -8  78% 91% 67%   140 235 53  32% 32% 33%  0 0 0  1 0 1   -1 0 -1  
2   -7 -2 -5  70% 91% 50%  120 139 103  17% 23% 13%  1 1 0  4 3 1  -3 -2 -1  
3    -7 0 -7  65% 82% 50%    66 105 29  25% 19% 67%  0 0 0  2 1 1  -2 -1 -1  
4  -11 -5 -6  52% 64% 42%     70 56 83  29% 44% 23%  2 1 1  1 0 1   +1 +1 0  
  Unnamed: 0            R2.0          ACS        K            D      A  \
0  Derke VIT 

<ipython-input-352-17e2ae8b064b>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  individual_map_stats_team = pd.read_html(str(table))[0]


In [324]:
# IMPORTANT: we need labels on what teams are in each table.

# Successfully fetched each match's map data. 😸
# now clean data?
# implement for loops
# when cleaning data, check each table for multi leveled indexes, aka extra headers
# if extra headers, watch 18:40 of dataquest vid
# then lastly write to csv file? 35:00 in dataquest

# Note: may want to add time of matches to data
# so we know how recently a match was played for weighing stats or something, idk?

# I should really get more recent team map steps rather than all time which I can do.
# Will worry about in future?